In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import TextVectorization, Embedding, SimpleRNN, Dense, LSTM, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import tensorflow_hub as hub


from sklearn.model_selection import train_test_split
import gensim.downloader as gensim_downloader
import gensim
import multiprocessing
from mlflow import MlflowClient
import mlflow
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
from joblib import Parallel, delayed
from Source.preprocess_data import *  ## import all functions from preprocess_data.py
from Source.postprocess_data import * ## import all functions from postprocess_data.py
from Source.utils import *  ## import all functions from utils.py
import nltk
import optuna

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix 
from nltk.corpus import stopwords  
from nltk.tokenize import TweetTokenizer, WordPunctTokenizer, RegexpTokenizer
from nltk.stem import PorterStemmer, WordNetLemmatizer, LancasterStemmer, SnowballStemmer
from tqdm import tqdm

from transformers import AutoTokenizer, TFAutoModel
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from transformers import AutoTokenizer, TFAutoModel
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from transformers import AutoTokenizer, TFAutoModel
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers


tqdm.pandas()
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

nw = multiprocessing.cpu_count()



client = MlflowClient(tracking_uri="http://localhost:8080")
os.environ["TF_KERAS"]='1'
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("GPUs disponibles :", tf.config.list_physical_devices("GPU"))
print("Version TF :", tf.__version__)

c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\tensorflow_hub\__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version
c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.10.1
Num GPUs Available:  1
GPUs disponibles : [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Version TF : 2.10.1


In [2]:
df = pd.read_csv('https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/AI+Engineer/Project+7%C2%A0-+D%C3%A9tectez+les+Bad+Buzz+gr%C3%A2ce+au+Deep+Learning/sentiment140.zip',
                header=None,
                compression='zip',
                encoding='cp1252')

df.columns = ['target', 'ids', 'date', 'flag', 'user', 'text']

sample_df, _ = train_test_split(df, test_size=0.9, random_state=42, stratify=df['target'])
sample_df = sample_df.reset_index(drop=True)
print(f"Sample size: {sample_df.shape[0]} rows")
# On ne garde que les colonnes 'target' et 'text'
sample_df = sample_df[['target', 'text']]
sample_df["target"] = sample_df["target"].apply(lambda x: 0 if x == 0 else 1)
sample_df.to_csv('Data/raw_data.csv', index=False)


Sample size: 160000 rows


In [3]:
# Data
X_raw = sample_df['text']
y = sample_df['target']
X_train, X_val, y_train, y_val = train_test_split(X_raw, y, test_size=0.2, random_state=42, stratify=y)


In [4]:
X_train = X_train.to_list()
X_val = X_val.to_list()
y_train = y_train.to_list()
y_val = y_val.to_list()

# Universal sentence encoder 

In [5]:
# Création de l'étude Optuna et optimisation
print("Setting up MLflow experiment...")
mlflow.set_experiment("USE_models_experiment")
exp_id = mlflow.get_experiment_by_name("USE_models_experiment").experiment_id

experiment_description = (
    "Essai avec Universal sentence encoder "
    ""
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "Sentiment analysis modelling",
    "model_type": "RNN_types-pretrained-embeddings",
    "team": "Ph. Constant",
    "project_quarter": "Q3-2025",
    "mlflow.note.content": experiment_description,
}

for key, value in experiment_tags.items():
    client.set_experiment_tag(exp_id, key, value)

Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 367, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 465, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1635, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1628, in _read_helper
    result = read_yaml(root, file_name)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\utils\yaml_utils.py", line 107, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' d

Setting up MLflow experiment...


In [6]:
os.environ["TF_XLA_FLAGS"] = "--tf_xla_enable_xla_devices=false"


In [7]:
epochs = 50
lr = 1e-3


In [8]:
use = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
y_val   = tf.convert_to_tensor(y_val, dtype=tf.float32)

# Pré-calcul
# Pré-calcul embeddings sur CPU
def use_layer_experiment(dense_size):
     with mlflow.start_run():
        mlflow.log_params(params={
            'num_words':1,               
            'max_len': 1,
            'min_count': 1,
            'stemmer': 'None', 
            'latent_dim': 'None', 
            'rnn_size': 'None', 
            'epochs': epochs, 
            'learning_rate': lr,
            'embedding_name':'None',
            'rnn_layer_name':'USE embedding' 
        })
        model_savepath = "./Models/USE_model_exp.h5"


        with tf.device("/CPU:0"):
            X_train_emb = use(X_train)
            X_val_emb = use(X_val)

        

    # Classifieur simple
        model = Sequential([
            layers.Input(shape=(512,)),
            layers.Dense(dense_size, activation='relu', kernel_regularizer=regularizers.L2(l2=1e-4)),
            layers.Dropout(0.5),
            layers.Dense(dense_size, activation='relu', kernel_regularizer=regularizers.L2(l2=1e-4)),
            layers.Dropout(0.5),
            layers.Dense(1, activation='sigmoid')
        ])


        ## Callbacks
        checkpoint = ModelCheckpoint(model_savepath, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min')
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=10)
        lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=0, min_lr=1e-5)
        callbacks_list = [checkpoint, es, lr_scheduler]
        ## Compilation
        optimizer = Adam(learning_rate=lr)
        model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

        #Entrainement
        with tf.device("/GPU:0"):
            history = model.fit(X_train_emb, y_train, epochs=epochs, batch_size=64, validation_data=(X_val_emb,y_val), callbacks=callbacks_list, verbose=1)

        model.load_weights(model_savepath)

                # Prédictions sur le jeu de validation
        y_pred_proba = model.predict(X_val_emb)
        y_pred = (y_pred_proba>0.5)
        

        output_dict = postprocess_model_output(y_val, y_pred, y_pred_proba) # voir postprocess_data.py

        # Logging des métriques dans MLflow
        mlflow.log_metrics(output_dict)
        # Matrice de confusion
        cm = confusion_matrix(y_val, y_pred, normalize='pred')
        fig, ax = plt.subplots()
        sns.heatmap(cm, annot=True, fmt=".2f", cmap="Blues", ax=ax, )
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.title("Confusion Matrix - Validation Set")
        fig.savefig("confusion_matrix.png")
        plt.close(fig)
        mlflow.log_artifact("confusion_matrix.png")
        #
        fig2 = plot_training_history(history,show=False)
        fig2.savefig("learning_path.png")
        plt.close(fig2)
        mlflow.log_artifact("learning_path.png")

        # Enregistrement du modèle dans MLflow
        mlflow.tensorflow.log_model(model, "model")
        

In [9]:
print(f"Running test with USE layer")
use_layer_experiment(dense_size=128)

Running test with USE layer
Epoch 1/50
2000/2000 [==============================] - 7s 3ms/step - loss: 0.5043 - accuracy: 0.7734 - val_loss: 0.4720 - val_accuracy: 0.7862 - lr: 0.0010
Epoch 2/50
2000/2000 [==============================] - 7s 3ms/step - loss: 0.4794 - accuracy: 0.7864 - val_loss: 0.4670 - val_accuracy: 0.7888 - lr: 0.0010
Epoch 3/50
2000/2000 [==============================] - 7s 3ms/step - loss: 0.4746 - accuracy: 0.7893 - val_loss: 0.4648 - val_accuracy: 0.7911 - lr: 0.0010
Epoch 4/50
2000/2000 [==============================] - 7s 3ms/step - loss: 0.4718 - accuracy: 0.7916 - val_loss: 0.4647 - val_accuracy: 0.7914 - lr: 0.0010
Epoch 5/50
2000/2000 [==============================] - 7s 3ms/step - loss: 0.4708 - accuracy: 0.7926 - val_loss: 0.4648 - val_accuracy: 0.7926 - lr: 0.0010
Epoch 6/50
2000/2000 [==============================] - 6s 3ms/step - loss: 0.4698 - accuracy: 0.7936 - val_loss: 0.4649 - val_accuracy: 0.7922 - lr: 0.0010
Epoch 7/50
2000/2000 [========

2025/09/22 16:03:51 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/22 16:03:51 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmp0bq86xxk\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmp0bq86xxk\model\data\model\assets
2025/09/22 16:04:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


# Embedding MiniLM-L6-v2

In [18]:

model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModel.from_pretrained(model_name, from_pt=True)

def get_embeddings(texts, batch_size=128, max_length=128):
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        tokens = tokenizer(batch_texts, padding=True, truncation=True,
                           max_length=max_length, return_tensors="tf")
        outputs = model(**tokens)

        
        mean_embed = tf.reduce_mean(outputs.last_hidden_state, axis=1)  # pooling moyen
        # print(mean_embed.shape)
        max_embed = tf.reduce_max(outputs.last_hidden_state, axis=1) # pooling max
        # print(max_embed.shape)
        batch_emb = tf.concat([mean_embed, max_embed], axis=1)
        # print(batch_emb.shape)

        all_embeddings.append(batch_emb)
        # print(i)
    return tf.concat(all_embeddings, axis=0)

with tf.device("/CPU:0"):
            X_train_emb = get_embeddings(X_train) 
            X_val_emb = get_embeddings(X_val)


X_train_emb.shape

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


TensorShape([128000, 768])

In [30]:

y_train = np.array(y_train).astype("float32")
y_val   = np.array(y_val).astype("float32")
lr = 1e-4
epochs = 200
def minilm_layer_experiment(dense_size=128):
     with mlflow.start_run():
        mlflow.log_params(params={
            'num_words':1,               
            'max_len': 1,
            'min_count': 1,
            'stemmer': 'None', 
            'latent_dim': 'None', 
            'rnn_size': 'None', 
            'epochs': epochs, 
            'learning_rate': lr,
            'embedding_name':'None',
            'rnn_layer_name':'MiniLM embedding' 
        })
        model_savepath = "./Models/MiniLM_model_exp.h5"

        # Charger le modèle DistilBERT optimisé pour embeddings
        # model_name = "sentence-transformers/distilbert-base-nli-stsb-mean-tokens"
        # tokenizer = AutoTokenizer.from_pretrained(model_name)
        # model = TFAutoModel.from_pretrained(model_name)

        # def get_embeddings(texts, batch_size=32, max_length=128):
        #     all_embeddings = []
        #     for i in range(0, len(texts), batch_size):
        #         batch_texts = texts[i:i+batch_size]
        #         tokens = tokenizer(batch_texts, padding=True, truncation=True,
        #                    max_length=max_length, return_tensors="tf")
        #         outputs = model(**tokens)
        #         mean_embed = tf.reduce_mean(outputs.last_hidden_state, axis=1)  # pooling moyen
        #         max_embed = tf.reduce_max(outputs.last_hidden_state, axis=1) # pooling max
        #         batch_emb = tf.concat([mean_embed, max_embed], axis=1)
        #         all_embeddings.append(batch_emb)

        #     return tf.concat(all_embeddings, axis=0)

        # with tf.device("/CPU:0"):
        #     X_train_emb = get_embeddings(X_train)  
        #     X_val_emb = get_embeddings(X_val)




        model = Sequential([
            layers.Input(shape=(768,)),
            layers.Dense(dense_size*2, activation='relu', kernel_regularizer=regularizers.L2(l2=1e-3)),
            layers.Dropout(0.5),
            layers.Dense(dense_size, activation='relu', kernel_regularizer=regularizers.L2(l2=1e-3) ),
            layers.Dropout(0.5),
            layers.Dense(1, activation='sigmoid')
        ])

        ## Callbacks
        checkpoint = ModelCheckpoint(model_savepath, monitor='val_accuracy', verbose=0, save_best_only=True, save_weights_only=True, mode='max')
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=50)
        lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=20, verbose=0, min_lr=1e-5)
        callbacks_list = [checkpoint, es, lr_scheduler]
        ## Compilation
        optimizer = Adam(learning_rate=lr)
        model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

        #Entrainement
        with tf.device("/GPU:0"):
            history = model.fit(X_train_emb, y_train, epochs=epochs, batch_size=64, validation_data=(X_val_emb,y_val), callbacks=callbacks_list, verbose=1)

        model.load_weights(model_savepath)

                # Prédictions sur le jeu de validation
        y_pred_proba = model.predict(X_val_emb)
        y_pred = (y_pred_proba>0.5)
        

        output_dict = postprocess_model_output(y_val, y_pred, y_pred_proba) # voir postprocess_data.py

        # Logging des métriques dans MLflow
        mlflow.log_metrics(output_dict)
        # Matrice de confusion
        cm = confusion_matrix(y_val, y_pred, normalize='pred')
        fig, ax = plt.subplots()
        sns.heatmap(cm, annot=True, fmt=".2f", cmap="Blues", ax=ax, )
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.title("Confusion Matrix - Validation Set")
        fig.savefig("confusion_matrix.png")
        plt.close(fig)
        mlflow.log_artifact("confusion_matrix.png")
        #
        fig2 = plot_training_history(history,show=False)
        fig2.savefig("learning_path.png")
        plt.close(fig2)
        mlflow.log_artifact("learning_path.png")

        # Enregistrement du modèle dans MLflow
        mlflow.tensorflow.log_model(model, "model")

In [31]:
print(f"Running test with MiniLM layer")
minilm_layer_experiment(dense_size=128)

Running test with MiniLM layer
Epoch 1/200
2000/2000 [==============================] - 7s 3ms/step - loss: 0.9217 - accuracy: 0.7046 - val_loss: 0.7439 - val_accuracy: 0.7577 - lr: 1.0000e-04
Epoch 2/200
2000/2000 [==============================] - 5s 2ms/step - loss: 0.7012 - accuracy: 0.7514 - val_loss: 0.6380 - val_accuracy: 0.7636 - lr: 1.0000e-04
Epoch 3/200
2000/2000 [==============================] - 4s 2ms/step - loss: 0.6208 - accuracy: 0.7590 - val_loss: 0.5830 - val_accuracy: 0.7676 - lr: 1.0000e-04
Epoch 4/200
2000/2000 [==============================] - 5s 2ms/step - loss: 0.5800 - accuracy: 0.7622 - val_loss: 0.5567 - val_accuracy: 0.7693 - lr: 1.0000e-04
Epoch 5/200
2000/2000 [==============================] - 5s 2ms/step - loss: 0.5571 - accuracy: 0.7653 - val_loss: 0.5397 - val_accuracy: 0.7713 - lr: 1.0000e-04
Epoch 6/200
2000/2000 [==============================] - 5s 3ms/step - loss: 0.5415 - accuracy: 0.7685 - val_loss: 0.5288 - val_accuracy: 0.7711 - lr: 1.0000e-

2025/09/22 18:05:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/22 18:05:43 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmpa95f3lnk\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmpa95f3lnk\model\data\model\assets
2025/09/22 18:05:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


# Utilisation de DistilBert

In [33]:


# Charger le modèle DistilBERT optimisé pour embeddings
model_name = "sentence-transformers/distilbert-base-nli-stsb-mean-tokens"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModel.from_pretrained(model_name, from_pt=True)


with tf.device("/CPU:0"):
            X_train_emb = get_embeddings(X_train) 
            X_val_emb = get_embeddings(X_val)


X_train_emb.shape



Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\bassm\.cache\huggingface\hub\models--sentence-transformers--distilbert-base-nli-stsb-mean-tokens. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In orde

TensorShape([128000, 1536])

In [34]:
X_train_emb2 = X_train_emb 
X_val_emb2 = X_val_emb
y_train = np.array(y_train).astype("float32")
y_val   = np.array(y_val).astype("float32")
lr = 1e-4
epochs = 200
def db_nli_stsb_layer_experiment(dense_size=128):
     with mlflow.start_run():
        mlflow.log_params(params={
            'num_words':1,               
            'max_len': 1,
            'min_count': 1,
            'stemmer': 'None', 
            'latent_dim': 'None', 
            'rnn_size': 'None', 
            'epochs': epochs, 
            'learning_rate': lr,
            'embedding_name':'None',
            'rnn_layer_name':'DistilBERT-nli-stsb' 
        })
        model_savepath = "./Models/DB-NLI-STSB_model_exp.h5"




        model = Sequential([
            layers.Input(shape=(1536,)),
            layers.Dense(dense_size*2, activation='relu', kernel_regularizer=regularizers.L2(l2=1e-3)),
            layers.Dropout(0.5),
            layers.Dense(dense_size, activation='relu', kernel_regularizer=regularizers.L2(l2=1e-3) ),
            layers.Dropout(0.5),
            layers.Dense(1, activation='sigmoid')
        ])

        ## Callbacks
        checkpoint = ModelCheckpoint(model_savepath, monitor='val_accuracy', verbose=0, save_best_only=True, save_weights_only=True, mode='max')
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=50)
        lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=20, verbose=0, min_lr=1e-5)
        callbacks_list = [checkpoint, es, lr_scheduler]
        ## Compilation
        optimizer = Adam(learning_rate=lr)
        model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

        #Entrainement
        with tf.device("/GPU:0"):
            history = model.fit(X_train_emb, y_train, epochs=epochs, batch_size=64, validation_data=(X_val_emb,y_val), callbacks=callbacks_list, verbose=1)

        model.load_weights(model_savepath)

                # Prédictions sur le jeu de validation
        y_pred_proba = model.predict(X_val_emb)
        y_pred = (y_pred_proba>0.5)
        

        output_dict = postprocess_model_output(y_val, y_pred, y_pred_proba) # voir postprocess_data.py

        # Logging des métriques dans MLflow
        mlflow.log_metrics(output_dict)
        # Matrice de confusion
        cm = confusion_matrix(y_val, y_pred, normalize='pred')
        fig, ax = plt.subplots()
        sns.heatmap(cm, annot=True, fmt=".2f", cmap="Blues", ax=ax, )
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.title("Confusion Matrix - Validation Set")
        fig.savefig("confusion_matrix.png")
        plt.close(fig)
        mlflow.log_artifact("confusion_matrix.png")
        #
        fig2 = plot_training_history(history,show=False)
        fig2.savefig("learning_path.png")
        plt.close(fig2)
        mlflow.log_artifact("learning_path.png")

        # Enregistrement du modèle dans MLflow
        mlflow.tensorflow.log_model(model, "model")

In [35]:
print(f"Running test with MiniLM layer")
db_nli_stsb_layer_experiment(dense_size=128)

Running test with MiniLM layer
Epoch 1/200
2000/2000 [==============================] - 8s 4ms/step - loss: 0.9854 - accuracy: 0.7566 - val_loss: 0.8217 - val_accuracy: 0.7834 - lr: 1.0000e-04
Epoch 2/200
2000/2000 [==============================] - 7s 3ms/step - loss: 0.7406 - accuracy: 0.7787 - val_loss: 0.6548 - val_accuracy: 0.7859 - lr: 1.0000e-04
Epoch 3/200
2000/2000 [==============================] - 7s 3ms/step - loss: 0.6166 - accuracy: 0.7859 - val_loss: 0.5726 - val_accuracy: 0.7899 - lr: 1.0000e-04
Epoch 4/200
2000/2000 [==============================] - 7s 3ms/step - loss: 0.5574 - accuracy: 0.7891 - val_loss: 0.5316 - val_accuracy: 0.7918 - lr: 1.0000e-04
Epoch 5/200
2000/2000 [==============================] - 7s 3ms/step - loss: 0.5255 - accuracy: 0.7932 - val_loss: 0.5104 - val_accuracy: 0.7935 - lr: 1.0000e-04
Epoch 6/200
2000/2000 [==============================] - 7s 3ms/step - loss: 0.5071 - accuracy: 0.7946 - val_loss: 0.5052 - val_accuracy: 0.7886 - lr: 1.0000e-

2025/09/22 20:21:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/22 20:21:06 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmphsrv39wf\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmphsrv39wf\model\data\model\assets
2025/09/22 20:21:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
